In [77]:
!nvidia-smi

Tue Mar 18 09:54:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [78]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


# GRID

In [79]:
%%writefile grid.h
#ifndef GRID_H
#define GRID_H

#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include <assert.h>
#include <time.h>

#include <cuda_runtime.h>
#include <curand_kernel.h>

// Color definitions
#define RED 31
#define GREEN 32
#define YELLOW 33
#define BLUE 34
#define PURPLE 35

// Grid structure
typedef struct {
    int w;
    int h;
    int *g;
} Grid;

// Counter structure to track color counts
typedef struct {
    int cRED;
    int cYELLOW;
    int cBLUE;
    int cPURPLE;
} Counter;

// Initialize grid
static inline Grid GridInit(const int w, const int h) {
    assert(w > 0 && h > 0);
    Grid G = {w, h, (int*)malloc(sizeof(int) * w * h)};
    // Initialize grid to zeros
    memset(G.g, 0, sizeof(int) * w * h);
    return G;
}

// Free grid memory
static inline void freeGrid(Grid X) {
    free(X.g);
    X.g = NULL;
}

// Set grid value
static void GridSet(Grid *g, int x, int y, int value) {
    assert(g != NULL);
    g->g[y * g->w + x] = value;
}

// Display grid with ANSI colors
void GridDisplay(const Grid *g) {
    assert(g != NULL);

    const char *RESET = "\033[0m";
    const char *RED_COLOR = "\033[31m";
    const char *BLUE_COLOR = "\033[34m";
    const char *PURPLE_COLOR = "\033[35m";
    const char *YELLOW_COLOR = "\033[33m";

    for (int y = 0; y < g->h; y++) {
        for (int x = 0; x < g->w; x++) {
            int cell = g->g[y * g->w + x];
            if (cell == 0) {
                printf("  ");
            } else if (cell == RED) {
                printf("%sR %s", RED_COLOR, RESET);
            } else if (cell == BLUE) {
                printf("%sB %s", BLUE_COLOR, RESET);
            } else if (cell == PURPLE) {
                printf("%sP %s", PURPLE_COLOR, RESET);
            } else if (cell == YELLOW) {
                printf("%sY %s", YELLOW_COLOR, RESET);
            } else {
                printf("? ");
            }
        }
        printf("\n");
    }
}

// Print counter statistics
void printCounter(Counter x) {
    printf("__STATS___\n");
    printf("RED: %d\n", x.cRED);
    printf("BLUE: %d\n", x.cBLUE);
    printf("PURPLE: %d\n", x.cPURPLE);
    printf("YELLOW: %d\n", x.cYELLOW);
}

// Export grid to PPM file
Counter export_grid_to_ppm(const Grid* g, int width, int height, const char *filename) {
    FILE *file = fopen(filename, "w");
    if (!file) {
        perror("Failed to open file");
        Counter error = {-1, -1, -1, -1};
        return error;
    }

    // PPM header
    fprintf(file, "P3\n%d %d\n255\n", width, height);

    Counter counter = {0};

    // Map grid values to RGB colors
    for (int y = 0; y < height; y++) {
        for (int x = 0; x < width; x++) {
            int cell = g->g[y * width + x];
            int r = 173, g_val = 216, b = 230; // Default: light blue

            if (cell == RED) {      // Red
                r = 255; g_val = 0; b = 0;
                counter.cRED++;
            } else if (cell == BLUE) { // Blue
                r = 0; g_val = 0; b = 255;
                counter.cBLUE++;
            } else if (cell == PURPLE) { // Purple
                r = 128; g_val = 0; b = 128;
                counter.cPURPLE++;
            } else if (cell == YELLOW) { // Yellow
                r = 255; g_val = 255; b = 0;
                counter.cYELLOW++;
            }

            // Write RGB values to file
            fprintf(file, "%d %d %d ", r, g_val, b);
        }
        fprintf(file, "\n");
    }

    fclose(file);
    printf("Grid exported to %s\n", filename);
    return counter;
}

#endif // GRID_H

Overwriting grid.h


# walker

In [95]:
%%writefile walker.h

// walker.h
#ifndef WALKER_H
#define WALKER_H

#include <stdlib.h>
#include <curand_kernel.h>

// Array of colors structure
typedef struct {
    int clrs[4];
    int size;
} ArrClrs;

// Walker structure
typedef struct {
    int x;
    int y;
    int color;
    ArrClrs arrClrs;
    int active; // Flag to indicate if the walker is active
} Walker;

// Initialize walker
__host__ __device__ static Walker WalkerInit(int x, int y, int color) {
    Walker w = {x, y, color, {{0}, 0}, 1};
    return w;
}


// dla_cuda.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <curand_kernel.h>
#include <time.h>
#include "grid.h"
#include "walker.h"

// Error checking macro
#define cudaCheckError() { \
    cudaError_t e = cudaGetLastError(); \
    if (e != cudaSuccess) { \
        printf("CUDA error %s:%d: %s\n", __FILE__, __LINE__, cudaGetErrorString(e)); \
        exit(EXIT_FAILURE); \
    } \
}

// CUDA kernels and device functions

// Add a unique color to the array
__device__ void addUniqueColor(ArrClrs *arr, int color) {
    for (int i = 0; i < arr->size; ++i) {
        if (arr->clrs[i] == color) {
            return; // Color already exists
        }
    }
    if (arr->size < 4) { // Ensure we don't exceed array bounds
        arr->clrs[arr->size++] = color;
    }
}

// Check if the walker's color array contains a specific color
__device__ int isWalkerClrMatch(const Walker *w, int color) {
    for (int i = 0; i < w->arrClrs.size; ++i) {
        if (w->arrClrs.clrs[i] == color) {
            return 1;
        }
    }
    return 0;
}

// Check if a walker is adjacent to a cluster
__device__ int isAdjacentToCluster(const int *grid, int width, int height, Walker *w) {
    // Reset the Walker's color array
    w->arrClrs.size = 0;

    int x = w->x;
    int y = w->y;

    // Check Left
    if (x > 0 && grid[y * width + (x - 1)] != 0) {
        addUniqueColor(&w->arrClrs, grid[y * width + (x - 1)]);
    }
    // Check Right
    if (x < width - 1 && grid[y * width + (x + 1)] != 0) {
        addUniqueColor(&w->arrClrs, grid[y * width + (x + 1)]);
    }
    // Check Up
    if (y > 0 && grid[(y - 1) * width + x] != 0) {
        addUniqueColor(&w->arrClrs, grid[(y - 1) * width + x]);
    }
    // Check Down
    if (y < height - 1 && grid[(y + 1) * width + x] != 0) {
        addUniqueColor(&w->arrClrs, grid[(y + 1) * width + x]);
    }

    // Return the number of unique colors found
    return w->arrClrs.size;
}

// Move a walker randomly
__device__ void walkerMove(Walker *w, int *grid, int width, int height, curandState *state) {
    // Clear the old position of the walker in the grid (not needed if we're using atomic operations later)
    // grid[w->y * width + w->x] = 0;

    // Move the walker randomly
    int direction = curand(state) % 4; // 0=up, 1=down, 2=left, 3=right
    int newX = w->x;
    int newY = w->y;

    switch (direction) {
        case 0: if (w->y > 0) newY--; break; // Up
        case 1: if (w->y < height - 1) newY++; break; // Down
        case 2: if (w->x > 0) newX--; break; // Left
        case 3: if (w->x < width - 1) newX++; break; // Right
    }

    // Update the walker's position
    w->x = newX;
    w->y = newY;

    // No need to set the grid here, we'll do it atomically later if needed
}

// Place a walker on the grid
__device__ void walkerPlaceOnGrid(const Walker *w, int *grid, int width) {
    // Use atomic operation to prevent race conditions
    atomicExch(&grid[w->y * width + w->x], w->color);
}

// Spawn green walkers and create purple cells
__device__ void spawnGreen(Walker w, int *grid, int width, int height, int R, float p, curandState *state) {
    if (curand_uniform(state) >= p)
        return;

    int greenFlag = 0;
    int a = -1, b = -1;

    // Find an non-empty spot near the walker
    for (int i = -R; i <= R; i++) {
        for (int j = -R; j <= R; j++) {
            int nx = w.x + i;
            int ny = w.y + j;

            // Check if within bounds
            if (nx >= 0 && nx < width && ny >= 0 && ny < height) {
                int cellValue = grid[ny * width + nx];

                if (cellValue != 0) {  // Found a non-empty cell
                    greenFlag = 1;
                    a = nx;
                    b = ny;
                    i = R + 1;  // Force exit from outer loop
                    break;      // Exit inner loop
                }
            }
        }
    }

    if (greenFlag == 0)
        return;  // No valid location found

    Walker greenWalker = WalkerInit(a, b, GREEN);

    // Move the green walker inside the cluster
    int maxSteps = 100;  // Prevent infinite loops
    while (maxSteps--) {
        // Move the green walker
        int direction = curand(state) % 4;
        int nx = greenWalker.x;
        int ny = greenWalker.y;

        switch (direction) {
            case 0: if (ny > 0) ny--; break; // Up
            case 1: if (ny < height - 1) ny++; break; // Down
            case 2: if (nx > 0) nx--; break; // Left
            case 3: if (nx < width - 1) nx++; break; // Right
        }

        greenWalker.x = nx;
        greenWalker.y = ny;

        if (grid[greenWalker.y * width + greenWalker.x] == 0) {
            // Found an empty spot, turn the area around it into PURPLE
            for (int dx = -1; dx <= 1; dx++) {
                for (int dy = -1; dy <= 1; dy++) {
                    int nx = greenWalker.x + dx;
                    int ny = greenWalker.y + dy;
                    if (nx >= 0 && nx < width && ny >= 0 && ny < height) {
                        if (grid[ny * width + nx] == 0)
                            atomicExch(&grid[ny * width + nx], PURPLE);
                    }
                }
            }
            break;
        }
    }
}

// Detonate an area and change colors
__device__ void detonate(int *grid, int x, int y, int width, int height, int R, float pDet, float pPurp, curandState *state) {
    int detFlag = 0;

    if (curand_uniform(state) < pDet) {
        // Iterate over cells in radius R
        for (int i = -R; i <= R; i++) {
            for (int j = -R; j <= R; j++) {
                int nx = x + i;
                int ny = y + j;

                // Check if within bounds
                if (nx >= 0 && nx < width && ny >= 0 && ny < height) {
                    int cellValue = grid[ny * width + nx];

                    // Ensure YELLOW cells are not destroyed and destroy only opposing colors
                    if (cellValue != YELLOW) {
                        atomicExch(&grid[ny * width + nx], 0); // Destroy the cell
                        detFlag = 1;
                    }
                }
            }
        }
    }

    // Now handle the radius R+1 and change colors based on probability pPurp
    for (int i = -R - 1; i <= R + 1; i++) {
        for (int j = -R - 1; j <= R + 1; j++) {
            int nx = x + i;
            int ny = y + j;

            // Check if within bounds
            if (nx >= 0 && nx < width && ny >= 0 && ny < height) {
                int cellValue = grid[ny * width + nx];

                // If it's RED or BLUE, change to PURPLE with probability pPurp
                if ((cellValue == RED) || (cellValue == BLUE)) {
                    if (curand_uniform(state) < pPurp) {
                        // Change color to purple
                        atomicExch(&grid[ny * width + nx], PURPLE);
                    }
                }
            }
        }
    }

    // Destroy the cell at the center of the explosion
    if (detFlag) {
        atomicExch(&grid[y * width + x], 0);
    }
}

// Initialize random states
__global__ void initRandomStates(curandState *states, unsigned long seed) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    curand_init(seed, idx, 0, &states[idx]);
}

// Kernel for spawning and moving walkers
__global__ void dlaKernel(int *grid, int width, int height, Walker *walkers, int numWalkers, int maxSteps, curandState *states) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx >= numWalkers) return;

    curandState localState = states[idx];
    Walker walker = walkers[idx];

    // Only process active walkers
    if (!walker.active) return;

    // Choose walker color (alternating)
    int color = (idx % 2 == 0) ? RED : BLUE;

    // Spawn walker at edge
    int edge = curand(&localState) % 4;

    switch (edge) {
        case 0: // Top edge
            walker.x = curand(&localState) % width;
            walker.y = 0;
            break;
        case 1: // Bottom edge
            walker.x = curand(&localState) % width;
            walker.y = height - 1;
            break;
        case 2: // Left edge
            walker.x = 0;
            walker.y = curand(&localState) % height;
            break;
        case 3: // Right edge
            walker.x = width - 1;
            walker.y = curand(&localState) % height;
            break;
    }

    walker.color = color;

    // Random walk until maxSteps or interaction with grid
    for (int step = 0; step < maxSteps; step++) {
        walkerMove(&walker, grid, width, height, &localState);

        // Check if the walker is adjacent to any cluster
        int adjacent = isAdjacentToCluster(grid, width, height, &walker);

        if (adjacent) {
            if (isWalkerClrMatch(&walker, YELLOW) || isWalkerClrMatch(&walker, PURPLE) || isWalkerClrMatch(&walker, walker.color)) {
                // Same color: form a cluster
                walkerPlaceOnGrid(&walker, grid, width);

                // Spawn a green walker inside the cluster
                spawnGreen(walker, grid, width, height, 1, 0.01, &localState);
            } else {
                // Detonate at the current walker's position
                if (idx < numWalkers/2) {
                    detonate(grid, walker.x, walker.y, width, height, 1, 1.0 , 0.4, &localState);
                } else {
                    detonate(grid, walker.x, walker.y, width, height, 1, 0.6 , 0.1, &localState);
                }
            }

            // Mark walker as inactive
            walker.active = 0;
            break;
        }
    }

    // Save the walker state back
    walkers[idx] = walker;
    states[idx] = localState;
}


#endif // WALKER_H

Overwriting walker.h


# mp

In [96]:
%%writefile mp.cu
#include "grid.h"
#include "walker.h"


// Host function to run the DLA simulation
void runDLASimulation(Grid *grid, int maxWalkers, int maxSteps) {
    int width = grid->w;
    int height = grid->h;

    // Allocate device memory for grid
    int *d_grid;
    cudaMalloc(&d_grid, width * height * sizeof(int));
    cudaCheckError();

    // Copy grid data to device
    cudaMemcpy(d_grid, grid->g, width * height * sizeof(int), cudaMemcpyHostToDevice);
    cudaCheckError();

    //////////////////////////////////////////////////////////////////
    // Allocate memory for walkers
    // Walker *d_walkers;
    // cudaMalloc(&d_walkers, maxWalkers * sizeof(Walker));
    //cudaCheckError();


    Walker *d_walkers;
    cudaMalloc(&d_walkers, maxWalkers * sizeof(Walker));
    cudaCheckError();

    // Initialize walkers on host
    Walker *h_walkers = (Walker*)malloc(maxWalkers * sizeof(Walker));
    for (int i = 0; i < maxWalkers; i++) {
        // Initialize with dummy position and color; these will be reset in the kernel.
        h_walkers[i] = WalkerInit(0, 0, 0);
        h_walkers[i].active = 1;  // Mark the walker as active
    }

    // Copy initialized walkers to device memory
    cudaMemcpy(d_walkers, h_walkers, maxWalkers * sizeof(Walker), cudaMemcpyHostToDevice);
    cudaCheckError();
    free(h_walkers);
    //////////////////////////////////////////////////////////////////

    // Allocate memory for random states
    curandState *d_states;
    cudaMalloc(&d_states, maxWalkers * sizeof(curandState));
    cudaCheckError();

    // Initialize random states
    int blockSize = 256;
    int numBlocks = (maxWalkers + blockSize - 1) / blockSize;

    unsigned long seed = time(NULL);
    initRandomStates<<<numBlocks, blockSize>>>(d_states, seed);
    cudaCheckError();

    // Batch processing for walkers
    int batchSize = 1000; // Process 1000 walkers at a time
    int numBatches = (maxWalkers + batchSize - 1) / batchSize;

    for (int batch = 0; batch < numBatches; batch++) {
        int startIdx = batch * batchSize;
        int endIdx = min(startIdx + batchSize, maxWalkers);
        int currentBatchSize = endIdx - startIdx;

        // Calculate grid and block dimensions for this batch
        numBlocks = (currentBatchSize + blockSize - 1) / blockSize;

        // Launch kernel
        dlaKernel<<<numBlocks, blockSize>>>(d_grid, width, height,
                                            d_walkers + startIdx,
                                            currentBatchSize,
                                            maxSteps,
                                            d_states + startIdx);
        cudaCheckError();

        // Synchronize after each batch
        cudaDeviceSynchronize();

        // Progress update
        if (batch % 10 == 0 || batch == numBatches - 1) {
            printf("Processed batch %d/%d (walkers %d-%d)\n",
                   batch+1, numBatches, startIdx, endIdx-1);
        }
    }

    // Copy the grid back to host
    cudaMemcpy(grid->g, d_grid, width * height * sizeof(int), cudaMemcpyDeviceToHost);
    cudaCheckError();

    // Free device memory
    cudaFree(d_grid);
    cudaFree(d_walkers);
    cudaFree(d_states);
}

// Main function
int main() {
    int width = 300;
    int height = 300;
    int maxWalkers = 300*300;
    int maxSteps = 1000*1000; // Maximum steps per walker

    // Initialize grid
    Grid grid = GridInit(width, height);

    // Set initial seeds
    grid.g[(height / 2) * width + (width / 2)] = YELLOW;
    grid.g[(height / 2) * width + (width / 2) + 1] = YELLOW;

    // Run CUDA DLA simulation
    clock_t start = clock();
    runDLASimulation(&grid, maxWalkers, maxSteps);
    clock_t end = clock();

    double time_spent = (double)(end - start) / CLOCKS_PER_SEC;
    printf("CUDA simulation time: %.3f seconds\n", time_spent);

    // Export the final grid to PPM
    printf("Exporting grid:\n");
    Counter stats = export_grid_to_ppm(&grid, width, height, "./cuda_output.ppm");
    printCounter(stats);

    // Free host memory
    freeGrid(grid);

    return 0;
}

Overwriting mp.cu


# exec

In [97]:
!ls

cuda_output.ppm  grid.h  mp  mp.cu  sample_data  walker.h


In [98]:
!nvcc -arch=sm_75 mp.cu -o mp
!./mp

Processed batch 1/90 (walkers 0-999)
Processed batch 11/90 (walkers 10000-10999)
Processed batch 21/90 (walkers 20000-20999)
Processed batch 31/90 (walkers 30000-30999)
Processed batch 41/90 (walkers 40000-40999)
Processed batch 51/90 (walkers 50000-50999)
Processed batch 61/90 (walkers 60000-60999)
Processed batch 71/90 (walkers 70000-70999)
Processed batch 81/90 (walkers 80000-80999)
Processed batch 90/90 (walkers 89000-89999)
CUDA simulation time: 6.179 seconds
Exporting grid:
Grid exported to ./cuda_output.ppm
__STATS___
RED: 2550
BLUE: 2703
PURPLE: 9712
YELLOW: 2


In [99]:
from google.colab import files
files.download('./cuda_output.ppm')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>